In [ ]:
# import library
import tensorflow as tf
import numpy as np
import sys, os,cv2
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from skimage.transform import resize
from scipy.misc import imread
from imgaug import augmenters as iaa
import nibabel as nib
import imgaug as ia
from scipy.ndimage import zoom
from sklearn.utils import shuffle
from skimage import util 
import matplotlib.animation as animation

plt.style.use('seaborn-white')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
np.random.seed(6278)
tf.set_random_seed(6728)
ia.seed(6278)

# Generate training data
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# Get the certain STL 10 Data
train_data__location = "../../Dataset/STL10/stl10_binary/train_X.bin"; train_label_location = "../../Dataset/STL10/stl10_binary/train_y.bin"
test_data__location  = "../../Dataset/STL10/stl10_binary/test_X.bin";   test_label_location = "../../Dataset/STL10/stl10_binary/test_y.bin"
unlabeld_data_location = "../../Dataset/STL10/stl10_binary/unlabeled_X.bin";
# Code: https://github.com/mttk/STL10/blob/master/stl10_input.py
def read_all_images(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)

        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))

        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        return images
def read_labels(path_to_labels):
    """
    :param path_to_labels: path to the binary file containing labels from the STL-10 dataset
    :return: an array containing the labels
    """
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels
    
train_datas = read_all_images(train_data__location); train_label = read_labels(train_label_location)
test_datas  = read_all_images(test_data__location);    test_label = read_labels(test_label_location)
unlabel_data= read_all_images(unlabeld_data_location)
print(train_datas.shape,train_label.shape)
print(test_datas.shape, test_label.shape)
print(unlabel_data.shape)

In [ ]:
# show random images of STL data set and their classes
def show_images(data,label,col=10,row=10,color=True):
    fig=plt.figure(figsize=(15, 15))
    columns = col ; rows = row
    for i in range(1, columns*rows +1):
        fig.add_subplot(rows, columns, i)
        if color:  plt.imshow(data[i-1])
        else:      plt.imshow(data[i-1],cmap='gray')
        if not label is None: plt.title(str(label[i-1]))
        plt.grid(False)
        plt.axis('off')
    plt.show()
show_images(train_datas,train_label)

In [ ]:
# import layers
def tf_relu(x):   return tf.nn.relu(x)
def d_tf_relu(x): return tf.cast(tf.greater(x,0),tf.float64)


def tf_softmax_temp(x,temp=1.0):
    soften_version = tf.exp(x/temp) / tf.reduce_sum(tf.exp(x/temp), -1)
    return soften_version

class CNN():

    def __init__(self,k,inc,out, stddev=0.05,which_reg=0,act=tf_relu,d_act=d_tf_relu):
        self.w          = tf.Variable(tf.random_normal([k,k,inc,out],stddev=stddev,seed=2,dtype=tf.float64))
        self.b          = tf.Variable(tf.zeros(out,dtype=tf.float64))
        self.m,self.v   = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.mb,self.vb = tf.Variable(tf.zeros_like(self.b)),tf.Variable(tf.zeros_like(self.b))
        self.act,self.d_act = act,d_act
        self.which_reg  = which_reg
        
    def getw(self): return [self.w,self.b]

    def feedforward(self,input,stride=1,padding='VALID'):
        self.input  = input
        self.layer  = tf.nn.conv2d(input,self.w,strides=[1,stride,stride,1],padding=padding) + self.b
        self.layerA = self.act(self.layer)
        return self.layerA

    def backprop(self,gradient,stride=1,padding='VALID'):
        grad_part_1 = gradient
        grad_part_2 = self.d_act(self.layer)
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2

        grad = tf.nn.conv2d_backprop_filter(input = grad_part_3,filter_sizes = self.w.shape,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding) / batch_size
        grad_pass = tf.nn.conv2d_backprop_input(input_sizes = [batch_size] + list(grad_part_3.shape[1:]),
        filter= self.w,out_backprop = grad_middle,strides=[1,stride,stride,1],padding=padding)

        if self.which_reg == 0:   grad = grad
        if self.which_reg == 0.5: grad = grad + lamda * (tf.sqrt(tf.abs(self.w))) * (1.0/tf.sqrt(tf.abs(self.w)+ 10e-5)) * tf.sign(self.w)
        if self.which_reg == 1:   grad = grad + lamda * tf.sign(self.w)
        if self.which_reg == 1.5: grad = grad + lamda * 1.0/(tf.sqrt(tf.square(self.w) + 10e-5)) * self.w
        if self.which_reg == 2:   grad = grad + lamda * (1.0/tf.sqrt(tf.square(tf.abs(self.w))+ 10e-5)) * tf.abs(self.w) * tf.sign(self.w)
        if self.which_reg == 2.5: grad = grad + lamda * 2.0 * self.w
        if self.which_reg == 3:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),3)+ 10e-5,-0.66) * tf.pow(tf.abs(self.w),2) * tf.sign(self.w)
        if self.which_reg == 4:   grad = grad + lamda * tf.pow(tf.pow(tf.abs(self.w),4)+ 10e-5,-0.75) * tf.pow(tf.abs(self.w),3) * tf.sign(self.w)

        update_w = []
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1) ; v_hat = self.v / (1-beta2)
        adam_middel = learning_rate/(tf.sqrt(v_hat) + adam_e)
        update_w.append(tf.assign(self.w,tf.subtract(self.w,tf.multiply(adam_middel,m_hat)  )))
        return grad_pass,update_w

In [ ]:
# hyper parameters and layers
num_epoch = 100; learning_rate = 0.0008; batch_size = 10
beta1,beta2,adam_e = 0.9,0.999,1e-8

l1 = CNN(3,3,32)
l2 = CNN(4,32,64)
l3 = CNN(3,64,128)
l4 = CNN(3,128,256)
l5 = CNN(4,256,10)

In [ ]:
# create graph
x = tf.placeholder(shape=[batch_size,96,96,3],dtype=tf.float64)
temperature = tf.placeholder(shape=(),dtype=tf.float64)
y = tf.placeholder(shape=[batch_size,10],dtype=tf.float64)

layer1      = l1.feedforward(x)
layer1_pool = tf.nn.avg_pool(layer1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
layer2      = l2.feedforward(layer1_pool)
layer2_pool = tf.nn.avg_pool(layer2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
layer3      = l3.feedforward(layer2_pool)
layer3_pool = tf.nn.avg_pool(layer3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
layer4      = l4.feedforward(layer3_pool)
layer4_pool = tf.nn.avg_pool(layer4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
layer5      = l5.feedforward(layer4_pool)

final_prediction  = tf.squeeze(layer5)
print(final_prediction)

In [ ]:
# start the session
# sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess_results = sess.run(final_prediction,feed_dict={x:train_datas[:batch_size,:,:,:]})